In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_11 import *

## Serializing the model

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=2920)

In [3]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [5]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [5]:
len(il)

12954

In [26]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [7]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [4]:
def sched1_cycle(lr, pct_start=0.3, mom_start=.95, mom_mid=.85, mom_end=.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [43]:
lr = 1e-3
pct_start = 0.5
cbsched = sched1_cycle(lr, pct_start=pct_start)

In [15]:
learn.fit(40, cbs=cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.199050,0.212078,2.103301,0.251718,00:14
1,2.040197,0.301717,2.014710,0.311784,00:13
2,1.949357,0.344820,1.926632,0.351998,00:12
3,1.861118,0.386150,1.860408,0.377959,00:12
4,1.785314,0.419834,1.843209,0.406210,00:12
5,1.724389,0.456288,1.814801,0.409773,00:12
6,1.681516,0.471579,1.803301,0.425554,00:12
7,1.648793,0.488199,1.739576,0.446933,00:12
8,1.612661,0.508144,1.713331,0.469076,00:12
9,1.594549,0.514792,1.673064,0.473657,00:12


In [18]:
st = learn.model.state_dict()

In [19]:
type(st)

collections.OrderedDict

In [20]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [21]:
st['10.bias']


tensor([-0.0003, -0.0038, -0.0024,  0.0003, -0.0045, -0.0004,  0.0017, -0.0054,
         0.0012, -0.0066], device='cuda:0')

In [30]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

**It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.**

In [24]:
torch.save(st, mdl_path/'iw5')

## Pets

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3127)

In [6]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [26]:
pets.ls()

[PosixPath('/storage/data/oxford-iiit-pet/images'),
 PosixPath('/storage/data/oxford-iiit-pet/annotations')]

In [8]:
pets_path = pets/'images'

In [9]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [29]:
il

ImageList (7390 items)
[PosixPath('/storage/data/oxford-iiit-pet/images/american_bulldog_146.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/japanese_chin_12.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/Sphynx_247.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/beagle_158.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/beagle_76.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/shiba_inu_207.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/Siamese_56.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/keeshond_194.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/miniature_pinscher_89.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/leonberger_42.jpg')...]
Path: /storage/data/oxford-iiit-pet/images

In [10]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [11]:
random.seed(42)

In [12]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [13]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/storage/data/oxford-iiit-pet/images/american_bulldog_146.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/Sphynx_247.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/beagle_158.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/beagle_76.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/shiba_inu_207.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/Siamese_56.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/miniature_pinscher_89.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/beagle_18.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/wheaten_terrier_139.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/samoyed_157.jpg')...]
Path: /storage/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/storage/data/oxford-iiit-pet/images/japanese_chin_12.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/keeshond_194.jpg'), PosixPath('/storage/data/oxford-iiit-pet/images/leo

In [14]:
n = il.items[0].name; n

'american_bulldog_146.jpg'

In [15]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'american_bulldog'

In [16]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [17]:
proc = CategoryProcessor()

In [18]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [19]:
', '.join(proc.vocab)

'american_bulldog, Sphynx, beagle, shiba_inu, Siamese, miniature_pinscher, wheaten_terrier, samoyed, chihuahua, staffordshire_bull_terrier, Abyssinian, pug, British_Shorthair, Bombay, basset_hound, american_pit_bull_terrier, english_setter, pomeranian, Egyptian_Mau, yorkshire_terrier, keeshond, Russian_Blue, boxer, saint_bernard, Birman, english_cocker_spaniel, japanese_chin, Ragdoll, leonberger, Maine_Coon, newfoundland, havanese, german_shorthaired, Persian, Bengal, scottish_terrier, great_pyrenees'

In [20]:
ll.valid.x.tfms = val_tfms

In [27]:
c_out = len(proc.vocab)

In [42]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [43]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [44]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.540408,0.076346,3.436507,0.094053,00:18
1,3.299782,0.130493,3.410902,0.095436,00:10
2,3.083047,0.193340,3.043725,0.208852,00:10
3,2.790713,0.277336,2.806345,0.265560,00:10
4,2.552245,0.348133,2.598207,0.341632,00:10


## Custom head

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3265)

In [28]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [31]:
st = torch.load(mdl_path/'iw5')

In [32]:
m = learn.model

In [33]:
m.load_state_dict(st)

<All keys matched successfully>

In [34]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [35]:
xb, yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [36]:
pred = m_cut(xb)

In [37]:
pred.shape

torch.Size([128, 512, 4, 4])

In [38]:
ni = pred.shape[1]

In [39]:
# export
class AdapativeConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, xb):
        return torch.cat([self.mp(xb), self.ap(xb)], 1)

In [40]:
nh = 40
m_new = nn.Sequential(
    m_cut, AdapativeConcatPool2d(), Flatten(),
    nn.Linear(ni * 2, data.c_out)
)

In [41]:
learn.model = m_new

In [44]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.313787,0.716676,1.137190,0.744973,00:13
1,0.916362,0.840665,1.411872,0.638076,00:13
2,0.964507,0.815402,1.402944,0.645966,00:12
3,0.884057,0.852299,1.074969,0.774752,00:12
4,0.767412,0.910803,1.049870,0.778315,00:12


# adapt_model and gradual unfreezing

In [45]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb, yb = get_batch(data.train_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut,
        AdapativeConcatPool2d(), Flatten(),
        nn.Linear(ni * 2, data.c_out)
    )
    learn.model = m_new

In [46]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [47]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [48]:
for p in learn.model[0].parameters():
    p.requires_grad_(False)

In [49]:
learn.fit(3, sched1_cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.298520,0.198892,6.715285,0.103589,00:07
1,2.846141,0.219169,7.005295,0.107661,00:05
2,2.325949,0.297618,2.018160,0.326801,00:05


In [50]:
for p in learn.model[0].parameters():
    p.requires_grad_(True)

In [73]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.005727,0.219589,3.264474,0.160443,00:10
1,3.012603,0.216139,3.223257,0.165975,00:10
2,2.945949,0.235338,3.154473,0.204703,00:10
3,2.692027,0.313034,2.760524,0.307054,00:10
4,2.441469,0.406330,2.576779,0.373444,00:10


## Batch norm transfer

In [51]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [52]:
def apply_mod(m, f):
    f(m)
    for l in m.children():
        apply_mod(l, f)

def set_grad(m, b):
    if isinstance (m, (nn.Linear, nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters():
            p.requires_grad_(b)

In [53]:
apply_mod(learn.model, partial(set_grad, b=False))

In [54]:
learn.fit(3, sched1_cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.986406,0.830914,4.816325,0.143548,00:09
1,0.967900,0.861163,1.378488,0.674472,00:09
2,0.806587,0.894848,1.132685,0.756681,00:09


In [79]:
apply_mod(learn.model, partial(set_grad, b=True))

In [80]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.879770,0.594720,1.950317,0.582296,00:10
1,1.825291,0.608220,2.288650,0.439834,00:10
2,1.745881,0.625919,1.970441,0.538036,00:10
3,1.570719,0.707065,1.717573,0.659751,00:10
4,1.415623,0.775911,1.629884,0.690180,00:10


Pytorch already has an `apply` method we can use:

In [81]:
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and Param Groups

In [55]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func=opt_func, c_out=10, norm=norm_imagenette)

In [56]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [71]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d):
            g2 += l.parameters()
        elif hasattr(l, 'weight'):
            g1 += l.parameters()
        for ll in l.children():
            _bn_splitter(ll, g1, g2)
        
    g1, g2 = [], []
    _bn_splitter(m[0], g1, g2)
        
    g2 += m[1:].parameters()
    return g1, g2

In [72]:
a, b = bn_splitter(learn.model)

In [73]:
test_eq(len(a) + len(b), len(list(learn.model.parameters())))

In [74]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [75]:
# export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o: o for o in Learner.ALL_CBS})

In [76]:
cb_types.after_backward

'after_backward'

In [78]:
# export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None):
        self.cb_name = cb_name
        self.f = f
    def __call__(self, cb_name):
        if cb_name == self.cb_name:
            if self.f:
                self.f(self.run)
            else:
                set_trace()

In [79]:
# export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [80]:
disc_lr_sched = sched_1cycle([0, 3e-2], 0.5)

In [81]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [82]:
def _print_det(o): 
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [83]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.078748,0.799889,2.779968,0.376686,00:13
1,1.028755,0.805319,1.440130,0.658437,00:13
2,0.905822,0.849972,1.122915,0.766353,00:12


In [84]:
disc_lr_sched = sched_1cycle([1e-3, 1e-2], .3)

In [85]:
learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0001000000000000017, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.001000000000000017, 'sqr_mom': 0.99}]


In [86]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.837507,0.873684,1.291464,0.696869,00:13
1,1.014139,0.789917,1.350164,0.667345,00:12
2,0.980486,0.805319,1.238204,0.720285,00:12
3,0.860383,0.858726,1.100192,0.770171,00:12
4,0.760058,0.904820,1.079365,0.776533,00:12


# Export

In [87]:
!python ./notebook2script.py 11a_transfer_learning.ipynb

Converted 11a_transfer_learning.ipynb to exp/nb_11a.py
